[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/Security-Showcase/blob/main/notebooks/queryable_encryption_range.ipynb)




```
# This is formatted as code
```

# MongoDB Queryable Range Encryption with Atlas - Developer Education (8.0+ only)

This notebook is based on the available  [Queryable Encryption quick start](https://www.mongodb.com/docs/manual/core/queryable-encryption/quick-start/).

# Installation of prerequisites

In [1]:
!pip install pymongo[encryption]
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [2]:

import os
from pymongo import MongoClient
from pymongo.encryption import Algorithm, ClientEncryption, QueryType
from pymongo.encryption_options import AutoEncryptionOpts
from bson.codec_options import CodecOptions
from bson import json_util
import json
import requests
import platform
import tempfile

Function to download and set up crypt_shared library

In [4]:
def setup_crypt_shared():
    system = platform.system().lower()
    if system == "linux":
        url = "https://downloads.mongodb.com/linux/mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-8.0.0.tgz"
        filename = "mongo_crypt_shared_v1-linux-x86_64-enterprise-ubuntu2004-8.0.0.tgz"
    elif system == "darwin":
        url = "https://downloads.mongodb.com/osx/mongo_crypt_shared_v1-macos-x86_64-enterprise-8.0.0.tgz"
        filename = "mongo_crypt_shared_v1-macos-x86_64-enterprise-8.0.0.tgz"
    elif system == "windows":
        url = "https://downloads.mongodb.com/windows/mongo_crypt_shared_v1-windows-x86_64-enterprise-8.0.0.zip"
        filename = "mongo_crypt_shared_v1-windows-x86_64-enterprise-8.0.0.zip"
    else:
        raise OSError("Unsupported operating system")

    response = requests.get(url)
    response.raise_for_status()

    with tempfile.NamedTemporaryFile(delete=False, suffix=".tgz" if system != "windows" else ".zip") as tmp_file:
        tmp_file.write(response.content)
        tmp_file_path = tmp_file.name

    extract_dir = tempfile.mkdtemp()

    if system != "windows":
        os.system(f"tar -xzf {tmp_file_path} -C {extract_dir}")
        lib_path = os.path.join(extract_dir, "lib", "mongo_crypt_v1.so")
    else:
        os.system(f"powershell Expand-Archive -Path {tmp_file_path} -DestinationPath {extract_dir}")
        lib_path = os.path.join(extract_dir, "bin", "mongo_crypt_v1.dll")

    return lib_path

# Set up crypt_shared library

In [5]:
crypt_shared_lib_path = setup_crypt_shared()
print(f"Crypt shared library path: {crypt_shared_lib_path}")


Crypt shared library path: /tmp/tmp3sq3wf2j/lib/mongo_crypt_v1.so


 1. Connect to your Atlas cluster

In [6]:
import getpass
MONGODB_ATLAS_URI = getpass.getpass("Enter Atlas URI")

key_vault_client = MongoClient(MONGODB_ATLAS_URI, appname="deverel.content.python")

Enter Atlas URI··········


2. Set up encryption key and providers

In [7]:
local_master_key = os.urandom(96)
kms_providers = {"local": {"key": local_master_key}}
key_vault_namespace = "encryption.__keyVault"


In [8]:
kms_provider_name="local"
key_vault_database_name = "encryption"
key_vault_collection_name = "__keyVault"
key_vault_namespace = f"{key_vault_database_name}.{key_vault_collection_name}"
encrypted_database_name = "medicalRecords"
encrypted_collection_name = "patients"

3. Create a ClientEncryption object

In [9]:
!python3 -m pip install 'pymongo[encryption]'

In [10]:
auto_encryption_options = AutoEncryptionOpts(
    kms_providers,
    key_vault_namespace,
    crypt_shared_lib_path=crypt_shared_lib_path
)

 4. Set up the key vault

In [11]:
encrypted_client = MongoClient(
    MONGODB_ATLAS_URI, auto_encryption_opts=auto_encryption_options)


6. Define the encrypted fields map

In [15]:
encrypted_fields_map = {
    "fields": [
        {
            "path": "patientRecord.ssn",
            "bsonType": "int",
            "queries": [{"queryType": "range"}] # potentially 'range'
        },
        {
            "path": "patientRecord.billing",
            "bsonType": "object",
        }
    ]
}

7. Create an encrypted client

In [16]:
client_encryption = ClientEncryption(
    kms_providers=kms_providers,
    key_vault_namespace=key_vault_namespace,
    key_vault_client=encrypted_client,
    codec_options=CodecOptions()
)




 8. Set up and use the encrypted collection

In [17]:
client_encryption.create_encrypted_collection(
    encrypted_client[encrypted_database_name],
    encrypted_collection_name,
    encrypted_fields_map,
    kms_provider_name,
    {},
)

(Collection(Database(MongoClient(host=['80cluster-shard-00-00.u9i2v.mongodb.net:27017', '80cluster-shard-00-01.u9i2v.mongodb.net:27017', '80cluster-shard-00-02.u9i2v.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-zwqf6i-shard-0', tls=True, auto_encryption_opts=<pymongo.encryption_options.AutoEncryptionOpts object at 0x7d62cc0befb0>), 'medicalRecords'), 'patients'),
 {'fields': [{'path': 'patientRecord.ssn',
    'bsonType': 'int',
    'queries': [{'queryType': 'range'}],
    'keyId': Binary(b'\x86p\x17p\x16;G\xa9\xae\xcf\x80\xf3\x8az\xa3h', 4)},
   {'path': 'patientRecord.billing',
    'bsonType': 'object',
    'keyId': Binary(b')\xb5\x95\xd5%HL+\xad\xc1\x07\x9cX\xd6=O', 4)}]})

9. Insert an encrypted document

In [18]:
patient_document = {
    "patientName": "Jon Doe",
    "patientId": 12345678,
    "patientRecord": {
        "ssn": 987654320,
        "billing": {
            "type": "Visa",
            "number": "4111111111111111",
        },
    },
}
encrypted_collection = encrypted_client[encrypted_database_name][encrypted_collection_name]
result = encrypted_collection.insert_one(patient_document)
print(f"Inserted document ID: {result.inserted_id}")

Inserted document ID: 66fec5dc2c01cafbb97ba29e


10. Query the encrypted collection with a Range operator.

> Add blockquote



In [19]:
find_result = encrypted_collection.find_one({
    "patientRecord.ssn": {"$gte" : 987654320}
})
print(find_result)


{'_id': ObjectId('66fec5dc2c01cafbb97ba29e'), 'patientName': 'Jon Doe', 'patientId': 12345678, 'patientRecord': {'ssn': 987654320, 'billing': {'type': 'Visa', 'number': '4111111111111111'}}, '__safeContent__': [b'K\x17\xdf\x86\\\xa8\xe7\xd6Rt\x07A\x1e\xab\xccF\xcb9\n\x0f\x9e\xb5\xf1c\xe61{\xfe\xd9\xd0\xf9\x0b', b'A\x94\x0fo%rPk;\xa2f\x1b\x11}\xbb1:\x072+\x07\x96n\xb1\xaf\xcc\xc2SPb\xa68', b'y\x9e\x1am\x08H\x8c\x15\xd3;(\xe3\x12|5\t#\xb4\x0e\xd7\xd6\xb0\x8a\x96\x07d\xcb~bYU\x17', b'\x93\x05\xf8\xb7\xadC\xb56G\xa1,\xf8!\xb4|O\x96i\x8fQ\x92\xd2\xf2P\xfc\x13O\x19\x19is\xf2', b'5\xa3\xac\xfd\x1d\xea\xd8\t\x8f\xf5M\xc6N\x8bA1\xf8\xed\xb1\xfb~\x1e5H\xf4\x0e\x88&\xa3@\xd0\x14', b'~\xd3\x03\x17\x9d9i\x85\x11\xae\x10\x15q\\\xfa\xca\x88S\x15\xa6\x8a\x9eB3*\xb1\xa7e\x07Q\xc6!', b'\x0b\x00\xd8\xc7\x0b\xac\xd4\xebLK`\x05\xa3\x17\xc2&\xb4\x0b\x94M\x15\xcc^\x12{g\x8e]\xc1\x9a\xf1\x96', b"\x80\x10\xcejD\xc6r\x8f\xceY\xc3\xad<.\xfdx6\xb4\x83o\xb0'ij\\J\xc1\x11\xce\x1c/\x1f", b'\xf1?\xf1\xa1E\xd2\xcc\x94

In [20]:
print("\nQuerying without encryption:")

reg_mongoclient = MongoClient(MONGODB_ATLAS_URI)

find_result = reg_mongoclient[encrypted_database_name][encrypted_collection_name].find_one({
    "patientRecord.ssn": {"$gte" : 987654320}
})
print(find_result)

all_docs = reg_mongoclient[encrypted_database_name][encrypted_collection_name].find()
print("\nAll documents in the collection:")
for doc in all_docs:
    print(doc)


Querying without encryption:
None

All documents in the collection:
{'_id': ObjectId('66fec5dc2c01cafbb97ba29e'), 'patientName': 'Jon Doe', 'patientId': 12345678, 'patientRecord': {'ssn': Binary(b'\x0f\x86p\x17p\x16;G\xa9\xae\xcf\x80\xf3\x8az\xa3h\x10\x0e\xb4m\x11ut\x8f\xa4$P\xe3Cq` \xd2\xfe5\xd1\xe9$\x8f\x15\xb0N\xc1\xa1\x87E\xee\xc9\x9d.\x0b\x00\xb4\x17\x11h\x86\'\xcf\xa8\'\x05\xc9&\x0fT\xc0Z0\x0c\x06e\xbc\xad\xdb\xf7E\xeb\x17_\xcb[m\xcf\xdd\xae\x03\xd4K\xca\xa6S\x0b,\xb1\xfe\x9dRvn\xdc\xb3\xc0\xd8.\x06,eo\xd2:\xecQ\xf5\x0b\xb7M\x12N\xda\xff\x1b\x90\xa9\'\xa1\x14\x86\x9721\x95\x95\xb9\xb2Tr\x00\xc0\xb0`\xb5\x1a\x19^\x16K\x17\xdf\x86\\\xa8\xe7\xd6Rt\x07A\x1e\xab\xccF\xcb9\n\x0f\x9e\xb5\xf1c\xe61{\xfe\xd9\xd0\xf9\x0b\xd2\x19\xce%n_p\xc3\x0b\xea\xf3\x0c4H^\xee\xea\x9c\xe9\xb4h\x88\r\xe2\xc4o&\xfe\xe1C<\x19\xc6\x06\xf9\xc9\xa9\xcbr\x17n\x12\xa1%\x05d\x11\x05\xa8s\xde#h\x8a\xcc\x1f,?\x83\n<\xec\xbc\xe8A\x94\x0fo%rPk;\xa2f\x1b\x11}\xbb1:\x072+\x07\x96n\xb1\xaf\xcc\xc2SPb\xa68\xcf\x04\r\xd